In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from keras import initializers

from src.model import *
from src.utils import load_data
from src.customs.custom_initializers import (
    ErdosRenyi,
    InputMatrix,
    RegularNX,
    WattsStrogatzNX,
)

# FIX: Standardize data
# BUG: transient 0
# TODO: use fit with 1 epoch and then reset model states
# NOTE: Separate system generation to other repo
# TODO: save dt in params.json

data_file = '/home/dionisio35/Documents/thesis/data/MG/17.0/MG_tau17.0_dt0.05_n250000_t-end12500.0_seed6623.csv'
transient = 1000
# FIX
train_length = 20000

input_scaling = 0.5 # input sigma
reservoir_degree = 8
spectral_radius = 1.04
reservoir_sigma = 0.4
rewiring = 1.0

# FIX
units = 6000
seed = 42
reservoir_activation = 'tanh'

regularization = 1e-6
leak_rate = 1.0

In [ ]:
(
    transient_data,
    train_data,
    train_target,
    forecast_transient_data,
    val_data,
    val_target,
) = load_data(
    data_file,
    transient,
    train_length,
    step=5
)

input_initializer = InputMatrix(
    sigma=input_scaling
)

input_bias_initializer = initializers.Zeros()

reservoir_initializer = WattsStrogatzNX(
    degree=reservoir_degree,
    spectral_radius=spectral_radius,
    rewiring_p=rewiring,
    sigma=reservoir_sigma,
)

model = generate_ESN(
    units,
    leak_rate,
    train_data.shape[-1],
    reservoir_activation,
    input_initializer,
    input_bias_initializer,
    reservoir_initializer,
)

In [ ]:
model.train(
    transient_data,
    train_data,
    train_target,
    regularization
)

In [ ]:
pred = model.forecast(
    1000,
    forecast_transient_data,
    val_data,
    val_target
)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(val_target[0][:1000])

In [ ]:
import matplotlib.pyplot as plt

dt=1
forecast_length = pred.shape[1]
features = pred.shape[-1]

_val_target = val_target[:, :forecast_length, :]

xvalues = np.arange(0, forecast_length) * dt

# Make each plot on a different axis
fig, axs = plt.subplots(features, 1, sharey=True, figsize=(20, 9.6))
fig.tight_layout()
fig.subplots_adjust(top=0.9, bottom=0.08, hspace=0.3)

fig.suptitle('ESN', fontsize=16)
fig.supxlabel('t')

# Make this if-else better TODO
if features == 1:
    axs.plot(xvalues, _val_target[0, :], label="target")
    axs.plot(xvalues, pred[0, :], label="prediction")
    axs.legend()
else:
    for i in range(features):
        axs[i].plot(xvalues, _val_target[0, :, i], label="target")
        axs[i].plot(xvalues, pred[0, :, i], label="prediction")
        axs[i].legend()


plt.show()